In [ ]:
!pip install git+https://github.com/huggingface/diffusers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-2_9qi_zs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-2_9qi_zs
  Resolved https://github.com/huggingface/diffusers to commit c2916175186e2b6d9c2d09b13a753cc47f5d9e19
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2679276 sha256=4a1c42d8c222b8a99eca16479d836ec5d05ef76a4f586f41ffe65533397eb592
  Stored in directory: /tmp/pip-ephem-wheel-cache-o617irs1/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers


In [ ]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 68688, done.
remote: Counting objects: 100% (491/491), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 68688 (delta 281), reused 308 (delta 164), pack-reused 68197 (from 1)
Receiving objects: 100% (68688/68688), 48.79 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (50699/50699), done.


In [ ]:
%cd /content/diffusers/examples/text_to_image
!pip install -r requirements.txt

/content/diffusers/examples/text_to_image
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached n

In [ ]:
import torch
torch.__version__

'2.3.1+cu121'

In [ ]:
!pip install transformers>=4.25.1

In [ ]:
!pip install multilingual-clip

In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
from torch import nn
from diffusers import StableDiffusionPipeline
from multilingual_clip import pt_multilingual_clip
from transformers import AutoTokenizer
import os
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp


In [ ]:
# Load the trained model
model_path = "/content/drive/MyDrive/Colab Notebooks/thesis/models/fine-tuned-stable-diffusion-v1-4-model"
mclip_model_name = "M-CLIP/LABSE-Vit-L-14"
mclip_model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(mclip_model_name)
mclip_tokenizer = AutoTokenizer.from_pretrained(mclip_model_name)
pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    text_encoder=mclip_model,
    tokenizer=mclip_tokenizer,
)
pipe.safety_checker = None
# Load the test dataset embeddings
save_path = '/content/drive/MyDrive/Colab Notebooks/thesis/embeddings/labse/labse_test_2016_embeddings.pth'
labse_test_2016_embeddings = torch.load(save_path)

# Directory to save the generated images and checkpoints
generated_images_dir = "/content/drive/MyDrive/Colab Notebooks/thesis/evaluation/sd2-v1-4/generated_images"
checkpoint_file = "/content/drive/MyDrive/Colab Notebooks/thesis/evaluation/sd2-v1-4/checkpoint.txt"
os.makedirs(generated_images_dir, exist_ok=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
embeddings_data = labse_test_2016_embeddings
negative_prompt = ""

# Define batch size
batch_size = 16

## Load checkpoint if it exists
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        start_index = int(f.read().strip())
else:
    start_index = 0

# Split data into batches
batches = [embeddings_data[i:i + batch_size] for i in range(0, len(embeddings_data), batch_size)]

# Process each batch sequentially with torch.no_grad() for inference
with torch.no_grad():
    for batch_index, batch in enumerate(tqdm(batches[start_index:], desc="Processing Batches", initial=start_index)):
        text_embeddings = torch.stack([entry['text_embedding'] for entry in batch]).to(dtype=torch.float16)
        negative_prompt_embeddings = mclip_model.forward([negative_prompt] * len(batch), mclip_tokenizer).to(dtype=torch.float16)
        # Generate images
        images = pipe(prompt_embeds=text_embeddings.unsqueeze(1),
                      negative_prompt_embeds=negative_prompt_embeddings.unsqueeze(1),
                      num_images_per_prompt=1, num_inference_steps=pipe.scheduler.num_inference_steps).images

        # Save generated images
        for img, entry in zip(images, batch):
            image_path = entry['image_path']
            save_path = os.path.join(generated_images_dir, f"{image_path}_generated.png")
            img.save(save_path)

        # Save checkpoint after processing the batch
        with open(checkpoint_file, 'w') as f:
            f.write(str(batch_index + start_index + 1))

print("Inference complete. Images saved.")

Processing Batches:   0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   2%|▏         | 1/63 [57:30<59:25:26, 3450.43s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   3%|▎         | 2/63 [1:54:50<58:21:55, 3444.51s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   5%|▍         | 3/63 [2:52:11<57:22:54, 3442.90s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Processing Batches:   6%|▋         | 4/63 [3:49:27<56:22:47, 3440.13s/it]

  0%|          | 0/50 [00:00<?, ?it/s]